In [ ]:
import pandas as pd
import os
from openslide import OpenSlide
import xml.etree.ElementTree as ET
import numpy as np
from PIL import Image

In [ ]:
src = '/home/kuki/Desktop/Synology/aging/data/svs'
xml_fns = [_ for _ in os.listdir(src) if _.endswith('xml')]
xml_pths = [os.path.join(src,_) for _ in xml_fns]

for xml in xml_pths[0:1]:
    if os.


